In [1]:
import pandas as pd 
import numpy as np
import sqlite3
import matplotlib as plt

## Basic Data Cleaning 

#### Looking at Sales Dataset First

In [2]:
df_sales = pd.read_csv("dubai_valuation.csv")
df_sales

,instance_date,actual_worth,row_status_code,property_type_en,property_sub_type_en,area_name_en,actual_area
0,30-04-2005,9750000.00,COMPLETED,Land,Commercial,Al Baraha,696.77
1,16-03-2009,39883500.00,COMPLETED,Land,Commercial,Al Barsha First,2470.20
2,18-04-2004,4000000.00,COMPLETED,Land,Warehouse,Ras Al Khor Industrial Second,4455.35
3,22-03-2006,3000000.00,COMPLETED,Land,Residential,Al Barsha Third,1393.55
4,8/5/05,26208000.00,COMPLETED,Land,Industrial,Al Garhoud,4869.60
...,...,...,...,...,...,...,...
83386,15-05-2025,2144443.00,COMPLETED,Unit,Flat,Al Merkadh,111.90
83387,15-05-2025,11000000.00,COMPLETED,Land,Residential,Nad Al Shiba Fourth,2322.58
83388,15-05-2025,2799996.00,COMPLETED,Land,Residential,Mirdif,929.03
83389,15-05-2025,2109414.72,COMPLETED,Unit,Flat,Marsa Dubai,77.66


In [3]:
#We are not dealing with Land only properties so remove all rows that have property type as land

df_sales = df_sales[df_sales["property_type_en"] != "Land"].reset_index(drop=True)
df_sales.index = range(1, len(df_sales) + 1)
df_sales

,instance_date,actual_worth,row_status_code,property_type_en,property_sub_type_en,area_name_en,actual_area
1,23-03-2010,2.200000e+08,COMPLETED,Building,Building,Burj Khalifa,18481.57
2,19-04-2010,1.800000e+06,COMPLETED,Building,Villa,Wadi Al Safa 6,232.48
3,11/2/10,1.400000e+06,COMPLETED,Building,Villa,Al Thanayah Fourth,172.31
4,15-02-2010,8.500000e+05,COMPLETED,Unit,Flat,Marsa Dubai,66.33
5,25-02-2010,7.500000e+05,COMPLETED,Unit,Flat,Marsa Dubai,85.38
...,...,...,...,...,...,...,...
24579,15-05-2025,1.573635e+06,COMPLETED,Unit,Flat,Al Khairan First,71.67
24580,15-05-2025,1.539247e+06,COMPLETED,Unit,Flat,Al Thanyah Fifth,110.28
24581,15-05-2025,2.144443e+06,COMPLETED,Unit,Flat,Al Merkadh,111.90
24582,15-05-2025,2.109415e+06,COMPLETED,Unit,Flat,Marsa Dubai,77.66


In [4]:
#I want to see what are all the property types we have right now
types_properties = df_sales["property_type_en"].value_counts()
sub_types_properties = df_sales["property_sub_type_en"].value_counts()
print(f"Property_type : \n{types_properties}")
print(f"Subtype_property : \n {sub_types_properties}")

Property_type : 
property_type_en
Unit        22267
Building     2316
Name: count, dtype: int64
Subtype_property : 
 property_sub_type_en
Flat                  16025
Office                 4581
Villa                  2262
Shop                    965
Hotel Apartment         354
Hotel Rooms             259
Building                 54
Warehouse                39
Workshop                 14
Sized Partition          11
Clinic                    7
Hotel                     7
Stacked Townhouses        3
Gymnasium                 2
Name: count, dtype: int64


In [5]:
'''
Since we are looking for properties for that correlate with office-commercial, normal living apartments, 
and luxuary apartments we can remove shop, warehoue, size partition, clinic, hotel,gymnasium ect.
'''
df_sales = df_sales[df_sales["property_sub_type_en"].isin(["Flat", "Office", "Villa", "Hotel Apartment", "Hotel Rooms", "Buildings"])].reset_index(drop = True)
#check if the other subcategories excluded
sub_types_properties = df_sales["property_sub_type_en"].value_counts()
print(f"Subtype_property : \n {sub_types_properties}")

Subtype_property : 
 property_sub_type_en
Flat               16025
Office              4581
Villa               2262
Hotel Apartment      354
Hotel Rooms          259
Name: count, dtype: int64


In [6]:
df_sales.index = range(1, len(df_sales) + 1)
df_sales

,instance_date,actual_worth,row_status_code,property_type_en,property_sub_type_en,area_name_en,actual_area
1,19-04-2010,1800000.00,COMPLETED,Building,Villa,Wadi Al Safa 6,232.48
2,11/2/10,1400000.00,COMPLETED,Building,Villa,Al Thanayah Fourth,172.31
3,15-02-2010,850000.00,COMPLETED,Unit,Flat,Marsa Dubai,66.33
4,25-02-2010,750000.00,COMPLETED,Unit,Flat,Marsa Dubai,85.38
5,25-03-2010,1350000.00,COMPLETED,Building,Villa,Al Thanayah Fourth,179.90
...,...,...,...,...,...,...,...
23477,15-05-2025,1573635.31,COMPLETED,Unit,Flat,Al Khairan First,71.67
23478,15-05-2025,1539247.12,COMPLETED,Unit,Flat,Al Thanyah Fifth,110.28
23479,15-05-2025,2144443.00,COMPLETED,Unit,Flat,Al Merkadh,111.90
23480,15-05-2025,2109414.72,COMPLETED,Unit,Flat,Marsa Dubai,77.66


In [7]:
#Another major issue is the instance_data is not consistent so will convert all to mm/dd/yyyy format.
df_sales["instance_date"] = pd.to_datetime(df_sales["instance_date"],errors = "coerce",infer_datetime_format=True) #Letting pandas determine which time format it is in
df_sales["instance_date"] = df_sales["instance_date"].dt.strftime("%m/%d/%Y")
df_sales

/var/folders/pk/cfn_h0j14zb_9gy053hsz8d40000gn/T/ipykernel_1990/429888122.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_sales["instance_date"] = pd.to_datetime(df_sales["instance_date"],errors = "coerce",infer_datetime_format=True) #Letting pandas determine which time format it is in
/var/folders/pk/cfn_h0j14zb_9gy053hsz8d40000gn/T/ipykernel_1990/429888122.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_sales["instance_date"] = pd.to_datetime(df_sales["instance_date"],errors = "coerce",infer_datetime_format=True) #Letting pandas determine which time format it is in


,instance_date,actual_worth,row_status_code,property_type_en,property_sub_type_en,area_name_en,actual_area
1,04/19/2010,1800000.00,COMPLETED,Building,Villa,Wadi Al Safa 6,232.48
2,NaN,1400000.00,COMPLETED,Building,Villa,Al Thanayah Fourth,172.31
3,02/15/2010,850000.00,COMPLETED,Unit,Flat,Marsa Dubai,66.33
4,02/25/2010,750000.00,COMPLETED,Unit,Flat,Marsa Dubai,85.38
5,03/25/2010,1350000.00,COMPLETED,Building,Villa,Al Thanayah Fourth,179.90
...,...,...,...,...,...,...,...
23477,05/15/2025,1573635.31,COMPLETED,Unit,Flat,Al Khairan First,71.67
23478,05/15/2025,1539247.12,COMPLETED,Unit,Flat,Al Thanyah Fifth,110.28
23479,05/15/2025,2144443.00,COMPLETED,Unit,Flat,Al Merkadh,111.90
23480,05/15/2025,2109414.72,COMPLETED,Unit,Flat,Marsa Dubai,77.66


In [8]:
#now lets look into status_code to see what work would be incomplete
df_sales_status_count = df_sales["row_status_code"].value_counts()
df_sales_status_count

row_status_code
COMPLETED    23463
COMMITTED       10
ENTERED          8
Name: count, dtype: int64

In [9]:
#We can see that most of the data points that are not corresponding to completed status are new project which makes senses projects 
#initiated in recent years (2023-2025) with a few null value (16.67%)
df_sales_status_not_completed = df_sales[df_sales["row_status_code"].isin(["COMMITTED","ENTERED"])].reset_index(drop=True)
df_sales_status_not_completed.index = range(1, len(df_sales_status_not_completed) + 1)
df_sales_status_not_completed

,instance_date,actual_worth,row_status_code,property_type_en,property_sub_type_en,area_name_en,actual_area
1,NaN,908473.0,ENTERED,Unit,Office,Business Bay,105.50
2,NaN,2669.0,ENTERED,Unit,Office,Al Thanyah Fifth,248.05
3,11/20/2023,3650000.0,ENTERED,Unit,Flat,Al Hebiah First,488.79
4,07/22/2024,1200000.0,ENTERED,Unit,Flat,Business Bay,77.03
5,09/30/2024,2500000.0,COMMITTED,Unit,Flat,Marsa Dubai,153.59
6,12/26/2024,55000000.0,COMMITTED,Building,Villa,World Islands,1699.43
7,11/19/2023,2500000.0,COMMITTED,Unit,Flat,Zaabeel Second,111.28
8,07/15/2024,3000000.0,ENTERED,Unit,Hotel Rooms,World Islands,144.09
9,12/22/2023,2150000.0,ENTERED,Building,Villa,Al Thanayah Fourth,207.08
10,04/23/2025,35.0,ENTERED,Unit,Flat,Al Barsha South Fourth,35.77


In [10]:
#The actual worth is in dirhams should convert that into USD since all the other countries actual worth will be in USD
exchange_rate = 3.67
df_sales["actual_worth"]/= exchange_rate
df_sales

,instance_date,actual_worth,row_status_code,property_type_en,property_sub_type_en,area_name_en,actual_area
1,04/19/2010,490463.215259,COMPLETED,Building,Villa,Wadi Al Safa 6,232.48
2,NaN,381471.389646,COMPLETED,Building,Villa,Al Thanayah Fourth,172.31
3,02/15/2010,231607.629428,COMPLETED,Unit,Flat,Marsa Dubai,66.33
4,02/25/2010,204359.673025,COMPLETED,Unit,Flat,Marsa Dubai,85.38
5,03/25/2010,367847.411444,COMPLETED,Building,Villa,Al Thanayah Fourth,179.90
...,...,...,...,...,...,...,...
23477,05/15/2025,428783.463215,COMPLETED,Unit,Flat,Al Khairan First,71.67
23478,05/15/2025,419413.384196,COMPLETED,Unit,Flat,Al Thanyah Fifth,110.28
23479,05/15/2025,584316.893733,COMPLETED,Unit,Flat,Al Merkadh,111.90
23480,05/15/2025,574772.403270,COMPLETED,Unit,Flat,Marsa Dubai,77.66


In [11]:
#Dealing with null values
# Drop rows with nulls in critical columns
df_sales.dropna(subset=["instance_date", "actual_worth", "property_type_en", "area_name_en"], inplace=True)
df_sales

# Strip and standardize text fields
text_cols = ["row_status_code", "property_type_en", "property_sub_type_en", "area_name_en"]
for col in text_cols:
    df_sales[col] = df_sales[col].astype(str).str.strip().str.title()

# Create price per sqft column
df_sales["worth_per_sqft"] = df_sales["actual_worth"] / df_sales["actual_area"]

df_sales.to_csv("dubai_clean_sales_price.csv")

#### Looking into 2025 rental dataset now

In [12]:
df_UAE_rental_csv = pd.read_csv("UAE_rental_gigasheet.csv")
df_UAE_rental_csv.index = range(1, len(df_UAE_rental_csv) + 1)
df_UAE_rental_csv

,Address,Rent,Beds,Baths,Type,Area_in_sqft,Rent_per_sqft,Rent_category,Frequency,Furnishing,Purpose,Posted_date,Age_of_listing_in_days,Location,City,Latitude,Longitude
1,"The Gate Tower 2, The Gate Tower, Shams Gate D...",124000,3,4,Apartment,1785,69.467787,Medium,Yearly,Unfurnished,For Rent,00:00.0,45,Al Reem Island,Abu Dhabi,24.493598,54.407841
2,"Water's Edge, Yas Island, Abu Dhabi",140000,3,4,Apartment,1422,98.452883,Medium,Yearly,Unfurnished,For Rent,00:00.0,44,Yas Island,Abu Dhabi,24.494022,54.607372
3,"Al Raha Lofts, Al Raha Beach, Abu Dhabi",99000,2,3,Apartment,1314,75.342466,Medium,Yearly,Furnished,For Rent,00:00.0,31,Al Raha Beach,Abu Dhabi,24.485931,54.600939
4,"Marina Heights, Marina Square, Al Reem Island,...",220000,3,4,Penthouse,3843,57.246942,High,Yearly,Unfurnished,For Rent,00:00.0,57,Al Reem Island,Abu Dhabi,24.493598,54.407841
5,"West Yas, Yas Island, Abu Dhabi",350000,5,7,Villa,6860,51.020408,High,Yearly,Unfurnished,For Rent,00:00.0,65,Yas Island,Abu Dhabi,24.494022,54.607372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73738,"Al Huboob 1, Al Salamah, Umm Al Quwain",14000,0,1,Apartment,419,33.412888,Low,Yearly,Unfurnished,For Rent,00:00.0,129,Al Salamah,Umm Al Quwain,25.493412,55.575994
73739,"Umm Al Quwain Marina, Umm Al Quwain",14000,0,1,Apartment,500,28.000000,Low,Yearly,Unfurnished,For Rent,00:00.0,129,Umm Al Quwain Marina,Umm Al Quwain,25.527959,55.606527
73740,"King Faisal Street, Umm Al Quwain",50000,3,4,Apartment,2000,25.000000,Low,Yearly,Unfurnished,For Rent,00:00.0,110,King Faisal Street,Umm Al Quwain,NaN,NaN
73741,"Al Maqtaa, Umm Al Quwain",37000,1,2,Apartment,989,37.411527,Low,Yearly,Unfurnished,For Rent,00:00.0,181,Al Maqtaa,Umm Al Quwain,NaN,NaN


In [13]:
#remove posted data since can't extra info from it
df_UAE_rental_csv = df_UAE_rental_csv.drop(columns=["Posted_date", "Age_of_listing_in_days"], errors='ignore')
#convert rent and rent_per_sqft from AED to USD
df_UAE_rental_csv["Rent"] /= exchange_rate
df_UAE_rental_csv["Rent_per_sqft"] /= exchange_rate
df_UAE_rental_csv


,Address,Rent,Beds,Baths,Type,Area_in_sqft,Rent_per_sqft,Rent_category,Frequency,Furnishing,Purpose,Location,City,Latitude,Longitude
1,"The Gate Tower 2, The Gate Tower, Shams Gate D...",33787.465940,3,4,Apartment,1785,18.928552,Medium,Yearly,Unfurnished,For Rent,Al Reem Island,Abu Dhabi,24.493598,54.407841
2,"Water's Edge, Yas Island, Abu Dhabi",38147.138965,3,4,Apartment,1422,26.826399,Medium,Yearly,Unfurnished,For Rent,Yas Island,Abu Dhabi,24.494022,54.607372
3,"Al Raha Lofts, Al Raha Beach, Abu Dhabi",26975.476839,2,3,Apartment,1314,20.529282,Medium,Yearly,Furnished,For Rent,Al Raha Beach,Abu Dhabi,24.485931,54.600939
4,"Marina Heights, Marina Square, Al Reem Island,...",59945.504087,3,4,Penthouse,3843,15.598622,High,Yearly,Unfurnished,For Rent,Al Reem Island,Abu Dhabi,24.493598,54.407841
5,"West Yas, Yas Island, Abu Dhabi",95367.847411,5,7,Villa,6860,13.902019,High,Yearly,Unfurnished,For Rent,Yas Island,Abu Dhabi,24.494022,54.607372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73738,"Al Huboob 1, Al Salamah, Umm Al Quwain",3814.713896,0,1,Apartment,419,9.104329,Low,Yearly,Unfurnished,For Rent,Al Salamah,Umm Al Quwain,25.493412,55.575994
73739,"Umm Al Quwain Marina, Umm Al Quwain",3814.713896,0,1,Apartment,500,7.629428,Low,Yearly,Unfurnished,For Rent,Umm Al Quwain Marina,Umm Al Quwain,25.527959,55.606527
73740,"King Faisal Street, Umm Al Quwain",13623.978202,3,4,Apartment,2000,6.811989,Low,Yearly,Unfurnished,For Rent,King Faisal Street,Umm Al Quwain,NaN,NaN
73741,"Al Maqtaa, Umm Al Quwain",10081.743869,1,2,Apartment,989,10.193877,Low,Yearly,Unfurnished,For Rent,Al Maqtaa,Umm Al Quwain,NaN,NaN


In [14]:
with sqlite3.connect("rental.db") as conn:
    # Drop first to avoid the “locked” DROP TABLE inside to_sql
    conn.execute("DROP TABLE IF EXISTS rental_data;")
    df_UAE_rental_csv.to_sql("rental_data", conn, if_exists="replace", index=False)
    conn.execute("DELETE FROM rental_data WHERE City NOT IN (?);", ("Dubai",))
    conn.commit()                          

    query1 = """
        SELECT
            City,
            COUNT(*)                         AS num_listings,
            ROUND(AVG(Rent), 0)              AS avg_rent,
            ROUND(AVG(Rent_per_sqft), 2)     AS avg_rent_psf,
            MIN(Rent)                        AS min_rent,
            MAX(Rent)                        AS max_rent
        FROM rental_data
        GROUP BY City;
    """

    query2 = """
        SELECT * FROM rental_data
    """
    result1 = pd.read_sql_query(query1, conn)
    result2 = pd.read_sql_query(query2,conn)
   

print(result1)
result2


    City  num_listings  avg_rent  avg_rent_psf  min_rent      max_rent
0  Dubai         34250   58138.0         36.04       0.0  1.498638e+07


,Address,Rent,Beds,Baths,Type,Area_in_sqft,Rent_per_sqft,Rent_category,Frequency,Furnishing,Purpose,Location,City,Latitude,Longitude
0,"Binghatti Heights, JVC District 10, Jumeirah V...",34059.945504,2,2,Apartment,1145,29.746677,Medium,Yearly,Unfurnished,For Rent,Jumeirah Village Circle (JVC),Dubai,25.055945,55.207465
1,"Seasons Community, JVC District 15, Jumeirah V...",13623.978202,1,2,Apartment,655,20.799967,Low,Yearly,Unfurnished,For Rent,Jumeirah Village Circle (JVC),Dubai,25.055945,55.207465
2,"Autumn 1 Block B, Autumn, Seasons Community, J...",24523.160763,1,2,Apartment,896,27.369599,Medium,Yearly,Furnished,For Rent,Jumeirah Village Circle (JVC),Dubai,25.055945,55.207465
3,"Socio Tower A, Socio, Dubai Hills Estate, Dubai",34059.945504,2,1,Apartment,720,47.305480,Medium,Yearly,Unfurnished,For Rent,Dubai Hills Estate,Dubai,25.126891,55.263874
4,"Eleganz by Danube, JVC District 14, Jumeirah V...",28610.354223,1,2,Apartment,965,29.648035,Medium,Yearly,Furnished,For Rent,Jumeirah Village Circle (JVC),Dubai,25.055945,55.207465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34245,"Umm Suqeim 2, Umm Suqeim, Dubai",122615.803815,6,7,Villa,5500,22.293783,High,Yearly,Unfurnished,For Rent,Umm Suqeim,Dubai,25.164759,55.218728
34246,"Azizi Riviera, Meydan One, Meydan City, Dubai",12261.307902,0,1,Apartment,316,38.801607,Low,Yearly,Unfurnished,For Rent,Meydan City,Dubai,25.154167,55.290669
34247,"Binghatti Crescent, JVC District 11, Jumeirah ...",25885.558583,1,2,Apartment,731,35.411161,Medium,Yearly,Unfurnished,For Rent,Jumeirah Village Circle (JVC),Dubai,25.055945,55.207465
34248,"Ewan Residence 1, Ewan Residence, Dubai Invest...",17711.171662,2,2,Apartment,1200,14.759310,Low,Yearly,Unfurnished,For Rent,Dubai Investment Park (DIP),Dubai,24.985928,55.175013


In [15]:
#min rating of 0 is suspicious seems like null so lets investigate that
#I want to see the buttom 10 rent cost to see if reasonable
query3 = """
SELECT *, COUNT(*) AS Count
FROM rental_data
GROUP BY Rent
ORDER BY Rent
LIMIT 30
"""
result3 = pd.read_sql_query(query3, conn)
result3

,Address,Rent,Beds,Baths,Type,Area_in_sqft,Rent_per_sqft,Rent_category,Frequency,Furnishing,Purpose,Location,City,Latitude,Longitude,Count
0,"Al Barsha South 2, Al Barsha South, Al Barsha,...",0.000000,5,2,Villa,10000,0.000000,Low,Yearly,Unfurnished,For Rent,Al Barsha,Dubai,25.096326,55.198402,14
1,"The Address Residences Dubai Opera, Downtown D...",79.019074,2,2,Apartment,1250,0.063215,Low,Yearly,Furnished,For Rent,Downtown Dubai,Dubai,25.186684,55.274705,1
2,"Al Baraha, Deira, Dubai",4632.152589,0,2,Apartment,559,8.286498,Low,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334,2
3,"Sonapur, Muhaisnah 2, Muhaisnah, Dubai",4768.392371,0,2,Apartment,450,10.596427,Low,Yearly,Unfurnished,For Rent,Muhaisnah,Dubai,25.263870,55.420790,1
4,"Muhaisnah 2, Muhaisnah, Dubai",4874.114441,0,2,Apartment,450,10.831365,Low,Yearly,Unfurnished,For Rent,Muhaisnah,Dubai,25.263870,55.420790,1
5,"Hardware Building, Naif, Deira, Dubai",4904.632153,0,2,Apartment,172,28.515303,Low,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334,1
6,"Al Nakheel Building, Naif, Deira, Dubai",5177.111717,0,1,Apartment,193,26.824413,Low,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334,1
7,"Tools Building, Naif, Deira, Dubai",5449.591281,0,2,Apartment,180,30.275507,Low,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334,2
8,"Naif, Deira, Dubai",5722.070845,0,2,Apartment,220,26.009413,Low,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334,1
9,"Naif, Deira, Dubai",5994.550409,0,2,Apartment,243,24.668932,Low,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334,1


In [26]:
#From researching the market more online the minimum usually for cheapest living in dubai is about 7k USD rent per year. However 
#the properties we aim to buy and also sell are not in the cheapest range it is mid range to luxuary so we will omit all below 25k USD per year in rent
query4 = """
SELECT * 
FROM rental_data
WHERE Rent >= 25000
ORDER BY Rent
"""
result4 = pd.read_sql_query(query4,conn)
result4

,Address,Rent,Beds,Baths,Type,Area_in_sqft,Rent_per_sqft,Rent_category,Frequency,Furnishing,Purpose,Location,City,Latitude,Longitude
0,"Chaimaa Avenue Residences, JVC District 13, Ju...",2.506785e+04,1,1,Apartment,690,36.330214,Medium,Yearly,Furnished,For Rent,Jumeirah Village Circle (JVC),Dubai,25.055945,55.207465
1,"Queue Point, Liwan, Dubai",2.506785e+04,2,2,Apartment,1800,13.926582,Medium,Yearly,Unfurnished,For Rent,Liwan,Dubai,25.110121,55.367699
2,"Meydan Avenue, Meydan City, Dubai",2.506785e+04,1,2,Apartment,1070,23.427895,Medium,Yearly,Unfurnished,For Rent,Meydan City,Dubai,25.154167,55.290669
3,"Al Jaddaf, Dubai",2.506785e+04,1,2,Apartment,940,26.667923,Medium,Yearly,Unfurnished,For Rent,Al Jaddaf,Dubai,25.207504,55.322428
4,"Hoor 18, Al Jaddaf, Dubai",2.506785e+04,1,2,Apartment,1000,25.067847,Medium,Yearly,Unfurnished,For Rent,Al Jaddaf,Dubai,25.214294,55.324569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24464,"Signature Villas Frond G, Signature Villas Pal...",1.634877e+06,8,2,Villa,9505,172.001829,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.100190,55.118535
24465,"Hor Al Anz East, Hor Al Anz, Deira, Dubai",2.179837e+06,2,2,Residential Building,135774,16.054889,High,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334
24466,"Frond J, Palm Jumeirah, Dubai",3.269755e+06,10,2,Villa,30776,106.243656,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.118088,55.134201
24467,"Signature Villas Frond H, Signature Villas Pal...",4.359673e+06,10,2,Villa,23745,183.603833,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.118088,55.134201


In [30]:
query_new_rent = """
    SELECT AVG(Rent) as average_rent_per_year
    FROM rental_data
"""
res = pd.read_sql_query(query_new_rent,conn)
res

,average_rent_per_year
0,58137.973142


In [28]:
df_UAE_rental_csv = result4.copy()
df_UAE_rental_csv

,Address,Rent,Beds,Baths,Type,Area_in_sqft,Rent_per_sqft,Rent_category,Frequency,Furnishing,Purpose,Location,City,Latitude,Longitude
0,"Chaimaa Avenue Residences, JVC District 13, Ju...",2.506785e+04,1,1,Apartment,690,36.330214,Medium,Yearly,Furnished,For Rent,Jumeirah Village Circle (JVC),Dubai,25.055945,55.207465
1,"Queue Point, Liwan, Dubai",2.506785e+04,2,2,Apartment,1800,13.926582,Medium,Yearly,Unfurnished,For Rent,Liwan,Dubai,25.110121,55.367699
2,"Meydan Avenue, Meydan City, Dubai",2.506785e+04,1,2,Apartment,1070,23.427895,Medium,Yearly,Unfurnished,For Rent,Meydan City,Dubai,25.154167,55.290669
3,"Al Jaddaf, Dubai",2.506785e+04,1,2,Apartment,940,26.667923,Medium,Yearly,Unfurnished,For Rent,Al Jaddaf,Dubai,25.207504,55.322428
4,"Hoor 18, Al Jaddaf, Dubai",2.506785e+04,1,2,Apartment,1000,25.067847,Medium,Yearly,Unfurnished,For Rent,Al Jaddaf,Dubai,25.214294,55.324569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24464,"Signature Villas Frond G, Signature Villas Pal...",1.634877e+06,8,2,Villa,9505,172.001829,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.100190,55.118535
24465,"Hor Al Anz East, Hor Al Anz, Deira, Dubai",2.179837e+06,2,2,Residential Building,135774,16.054889,High,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334
24466,"Frond J, Palm Jumeirah, Dubai",3.269755e+06,10,2,Villa,30776,106.243656,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.118088,55.134201
24467,"Signature Villas Frond H, Signature Villas Pal...",4.359673e+06,10,2,Villa,23745,183.603833,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.118088,55.134201


In [31]:
#checking for duplicate records and dropping them
df_UAE_rental_csv.drop_duplicates(inplace=True)
#checking for null values
print(df_UAE_rental_csv.isnull().sum())
df_UAE_rental_csv

Address          0
Rent             0
Beds             0
Baths            0
Type             0
Area_in_sqft     0
Rent_per_sqft    0
Rent_category    0
Frequency        0
Furnishing       0
Purpose          0
Location         0
City             0
Latitude         9
Longitude        9
dtype: int64


,Address,Rent,Beds,Baths,Type,Area_in_sqft,Rent_per_sqft,Rent_category,Frequency,Furnishing,Purpose,Location,City,Latitude,Longitude
0,"Chaimaa Avenue Residences, JVC District 13, Ju...",2.506785e+04,1,1,Apartment,690,36.330214,Medium,Yearly,Furnished,For Rent,Jumeirah Village Circle (JVC),Dubai,25.055945,55.207465
1,"Queue Point, Liwan, Dubai",2.506785e+04,2,2,Apartment,1800,13.926582,Medium,Yearly,Unfurnished,For Rent,Liwan,Dubai,25.110121,55.367699
2,"Meydan Avenue, Meydan City, Dubai",2.506785e+04,1,2,Apartment,1070,23.427895,Medium,Yearly,Unfurnished,For Rent,Meydan City,Dubai,25.154167,55.290669
3,"Al Jaddaf, Dubai",2.506785e+04,1,2,Apartment,940,26.667923,Medium,Yearly,Unfurnished,For Rent,Al Jaddaf,Dubai,25.207504,55.322428
4,"Hoor 18, Al Jaddaf, Dubai",2.506785e+04,1,2,Apartment,1000,25.067847,Medium,Yearly,Unfurnished,For Rent,Al Jaddaf,Dubai,25.214294,55.324569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24464,"Signature Villas Frond G, Signature Villas Pal...",1.634877e+06,8,2,Villa,9505,172.001829,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.100190,55.118535
24465,"Hor Al Anz East, Hor Al Anz, Deira, Dubai",2.179837e+06,2,2,Residential Building,135774,16.054889,High,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334
24466,"Frond J, Palm Jumeirah, Dubai",3.269755e+06,10,2,Villa,30776,106.243656,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.118088,55.134201
24467,"Signature Villas Frond H, Signature Villas Pal...",4.359673e+06,10,2,Villa,23745,183.603833,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.118088,55.134201


In [32]:
#remove small records cause null values can be dangerous in future when using in tableau
df_UAE_rental_csv.dropna(subset=["Latitude", "Longitude"], inplace=True)

In [33]:
#Fix inconsistent casing and whitespaces
cols_to_clean = ["Type", "Furnishing", "Purpose", "Location", "City", "Frequency", "Rent_category"]
for col in cols_to_clean:
    df_UAE_rental_csv[col] = df_UAE_rental_csv[col].str.strip().str.title()
df_UAE_rental_csv

,Address,Rent,Beds,Baths,Type,Area_in_sqft,Rent_per_sqft,Rent_category,Frequency,Furnishing,Purpose,Location,City,Latitude,Longitude
0,"Chaimaa Avenue Residences, JVC District 13, Ju...",2.506785e+04,1,1,Apartment,690,36.330214,Medium,Yearly,Furnished,For Rent,Jumeirah Village Circle (Jvc),Dubai,25.055945,55.207465
1,"Queue Point, Liwan, Dubai",2.506785e+04,2,2,Apartment,1800,13.926582,Medium,Yearly,Unfurnished,For Rent,Liwan,Dubai,25.110121,55.367699
2,"Meydan Avenue, Meydan City, Dubai",2.506785e+04,1,2,Apartment,1070,23.427895,Medium,Yearly,Unfurnished,For Rent,Meydan City,Dubai,25.154167,55.290669
3,"Al Jaddaf, Dubai",2.506785e+04,1,2,Apartment,940,26.667923,Medium,Yearly,Unfurnished,For Rent,Al Jaddaf,Dubai,25.207504,55.322428
4,"Hoor 18, Al Jaddaf, Dubai",2.506785e+04,1,2,Apartment,1000,25.067847,Medium,Yearly,Unfurnished,For Rent,Al Jaddaf,Dubai,25.214294,55.324569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24464,"Signature Villas Frond G, Signature Villas Pal...",1.634877e+06,8,2,Villa,9505,172.001829,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.100190,55.118535
24465,"Hor Al Anz East, Hor Al Anz, Deira, Dubai",2.179837e+06,2,2,Residential Building,135774,16.054889,High,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334
24466,"Frond J, Palm Jumeirah, Dubai",3.269755e+06,10,2,Villa,30776,106.243656,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.118088,55.134201
24467,"Signature Villas Frond H, Signature Villas Pal...",4.359673e+06,10,2,Villa,23745,183.603833,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.118088,55.134201


In [34]:
#some feature engineering 
df_UAE_rental_csv["Rent_per_bed"] = df_UAE_rental_csv["Rent"] / df_UAE_rental_csv["Beds"].replace(0, 1)
df_UAE_rental_csv["Rent_per_bath"] = df_UAE_rental_csv["Rent"] / df_UAE_rental_csv["Baths"].replace(0, 1)
df_UAE_rental_csv.index = range(1, len(df_UAE_rental_csv) + 1)
df_UAE_rental_csv

,Address,Rent,Beds,Baths,Type,Area_in_sqft,Rent_per_sqft,Rent_category,Frequency,Furnishing,Purpose,Location,City,Latitude,Longitude,Rent_per_bed,Rent_per_bath
1,"Chaimaa Avenue Residences, JVC District 13, Ju...",2.506785e+04,1,1,Apartment,690,36.330214,Medium,Yearly,Furnished,For Rent,Jumeirah Village Circle (Jvc),Dubai,25.055945,55.207465,2.506785e+04,2.506785e+04
2,"Queue Point, Liwan, Dubai",2.506785e+04,2,2,Apartment,1800,13.926582,Medium,Yearly,Unfurnished,For Rent,Liwan,Dubai,25.110121,55.367699,1.253392e+04,1.253392e+04
3,"Meydan Avenue, Meydan City, Dubai",2.506785e+04,1,2,Apartment,1070,23.427895,Medium,Yearly,Unfurnished,For Rent,Meydan City,Dubai,25.154167,55.290669,2.506785e+04,1.253392e+04
4,"Al Jaddaf, Dubai",2.506785e+04,1,2,Apartment,940,26.667923,Medium,Yearly,Unfurnished,For Rent,Al Jaddaf,Dubai,25.207504,55.322428,2.506785e+04,1.253392e+04
5,"Hoor 18, Al Jaddaf, Dubai",2.506785e+04,1,2,Apartment,1000,25.067847,Medium,Yearly,Unfurnished,For Rent,Al Jaddaf,Dubai,25.214294,55.324569,2.506785e+04,1.253392e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21563,"Signature Villas Frond G, Signature Villas Pal...",1.634877e+06,8,2,Villa,9505,172.001829,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.100190,55.118535,2.043597e+05,8.174387e+05
21564,"Hor Al Anz East, Hor Al Anz, Deira, Dubai",2.179837e+06,2,2,Residential Building,135774,16.054889,High,Yearly,Unfurnished,For Rent,Deira,Dubai,25.272794,55.305334,1.089918e+06,1.089918e+06
21565,"Frond J, Palm Jumeirah, Dubai",3.269755e+06,10,2,Villa,30776,106.243656,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.118088,55.134201,3.269755e+05,1.634877e+06
21566,"Signature Villas Frond H, Signature Villas Pal...",4.359673e+06,10,2,Villa,23745,183.603833,High,Yearly,Furnished,For Rent,Palm Jumeirah,Dubai,25.118088,55.134201,4.359673e+05,2.179837e+06


In [24]:
#df_UAE_rental_csv.to_csv("dubai_cleaned_rental_price.csv")